In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow
from shapely.geometry import Point, Polygon, MultiPolygon
import shapely
import numba
import pyarrow as pa
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster


In [2]:
sectors = pd.read_csv("../data/NYPD_Sectors_20240306.csv").convert_dtypes(dtype_backend='pyarrow')
nypdf = pd.read_feather('../data/2018_tf.feather')
npydf = dd.from_pandas(nypdf, npartitions=20)
nypdf = nypdf[nypdf['agency']=="NYPD"]
nypdf = nypdf.dropna(how='any',subset=['latitude','longitude'])


In [3]:
sectors['the_geom'] = sectors['the_geom'].apply(
    lambda x:
        shapely.from_wkt(x)
)
sectors

,sector,pct,the_geom,phase,patrol_bor,SCT_TEXT,START_DATE,sq_miles,SQ_Feet,SctrFloat,Sq_MileNew
0,102A,102,MULTIPOLYGON (((-73.84119605660885 40.68335136...,14,PBQS,102A,07/09/2018 12:00:00 AM +0000,1.408760,3.927381e+07,0,1.408754
1,102B,102,MULTIPOLYGON (((-73.84451063791278 40.68241508...,14,PBQS,102B,07/09/2018 12:00:00 AM +0000,1.054450,2.939626e+07,1,1.054446
2,102C,102,MULTIPOLYGON (((-73.82610502096314 40.71556833...,14,PBQS,102C,07/09/2018 12:00:00 AM +0000,0.852163,2.375686e+07,2,0.852160
3,102D,102,MULTIPOLYGON (((-73.83680747734981 40.70844004...,14,PBQS,102D,07/09/2018 12:00:00 AM +0000,1.466645,4.088754e+07,3,1.466639
4,104A,104,MULTIPOLYGON (((-73.8584450472624 40.705782787...,15,PBQN,104A,10/01/2018 12:00:00 AM +0000,1.596795,4.451591e+07,4,2.194020
...,...,...,...,...,...,...,...,...,...,...,...
297,61C,61,MULTIPOLYGON (((-73.9360940736246 40.603873161...,12,PBBS,061C,01/22/2018 12:00:00 AM +0000,0.907510,2.529982e+07,340,0.907506
298,61D,61,MULTIPOLYGON (((-73.93182550005434 40.59299110...,12,PBBS,061D,01/22/2018 12:00:00 AM +0000,1.772059,4.940196e+07,341,1.772051
299,120A,120,MULTIPOLYGON (((-74.08777048902891 40.63171915...,3,PBSI,120A,12/14/2015 12:00:00 AM +0000,1.853173,5.166329e+07,336,1.853165
300,120B,120,MULTIPOLYGON (((-74.07337342888036 40.63661072...,3,PBSI,120B,12/14/2015 12:00:00 AM +0000,1.126137,3.139478e+07,337,1.126133


In [4]:



# @numba.jit(forceobj=True)
class Tracker:
    def __init__(self) -> None:
        self.count = 0

    def get_district(self, latitude:pa.float64, longitude:pa.float64, sv):
        self.count += 1
        dist = np.apply_along_axis(lambda x: x[0] if x[1].contains(Point(longitude,latitude)) else '', 1, sv)
        ret = dist[dist != '']
        if self.count % 1000 == 0:
            print(f"Row {self.count} is in district {ret}")
        if len(ret) == 0:
            return pa.NA
        return ret[0]

    # for _, sector in sectors[['sector','the_geom']].iterrows():
    #     # print(sector)
    #     mulpoly = sector['the_geom']

    #     # for poly in mulpoly:
    #     if mulpoly.contains(loc_point):
    #         ret = sector['sector']
    #         break


    # return ret
sv:np.ndarray = sectors[['sector','the_geom']].values

# get_district(*nypdf[['latitude', 'longitude']].iloc[0], sv)

In [5]:
def get_district(latitude:pa.float64, longitude:pa.float64, sv):
    dist = np.apply_along_axis(lambda x: x[0] if x[1].contains(Point(longitude,latitude)) else '', 1, sv)
    ret = dist[dist != '']
    if len(ret) == 0:
        return pa.NA
    return ret[0]

In [6]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=4, threads_per_worker=2)

client = Client(cluster)
# tracker = Tracker()
nypdf['precinct'] = nypdf[['latitude', 'longitude']].apply(lambda x: get_district(x['latitude'], x['longitude'], sv), axis=1)


KeyboardInterrupt: 

In [ ]:
assert False
test = sectors["the_geom"]

test.apply(
    lambda x:
        shapely.from_wkt(x)
)

In [ ]:


test = sectors["the_geom"]

test = test.apply(
    lambda x:
    MultiPolygon(
        [
            Polygon(
                [
                    [np.float64(num) for num in pair.lstrip(' ').split(' ')]
                    for pair in polygon.split(',')
                ]
            )
        for polygon in x.lstrip('MULTIPOLYGON (((').rstrip(')))').split(')), ((')
        ]
    )
)
MultiPolygon

# for i, val in test.items():
#     polygons =
#     # display(polygons)
#     (polygons)
#     # try:
    #     Polygon(
    #     [
    #         [np.float64(num) for num in t.lstrip(' ').split(' ')]
    #         for t in val.lstrip('MULTIPOLYGON (((').rstrip(')))').split(',')
    #         ]
    #     )
    # except ValueError as e:
    #     print(f"Row {i} error")
    #     print(val)
    #     # print(val[:20], val[-20:])
    #     raise e

In [ ]:
test

In [ ]:
Polygon(test)